# The Iterative K-means -/+ algorithm  
As seen in Ismkahn (2018)  
Coded by Henry Morgan, ULB Data Science

Note: Ismkahn's original C++ code is available on his researchgate profile, but this code is independently generated and may differ in implementation strategies

In [ ]:
import numpy as np

## Center and Point classes

### Center

In [ ]:
class center:
# a class to capture the required attributes of a cluster for the algorithms

    __slots__ = 'xyz', 'abc'

# Need: uniqueID
# Need: list of associated points
# Need: Center coordinates
# Need: Gain/Loss
# Need: SSEDM of cluster
# Need: adjacent clusters
# Need: indivisible marker
# Need: irremovable marker
    
    def __init__(self, coordinates,index):
    # initialisation requires an NP array holding the center's starting coordinates and the initialisation index of the center
        self.loc = coordinates
        self.id = index
    
    def getID()
    
    def getLoc()
    
    def getPoints()
    


### Point

In [ ]:
__slots__

# Need: uniqueID 
# Need: coordinates
# Need: First and Second closest centers
# Need: Distance to first and second closest centers
# Need: Useful Nearest Centers
    def getLoc()
    
    def getCenter()
    
    def disToFirst()
    
    def disToSecond()
    #distance to second nearest center
    
    def getSecondCenter()

## Useful Nearest Center initialisation algorithm

In [5]:
def distance(X1, X2):
# Function to calculate euclidean distance between two points or two centers
# Accepts two point or center class variables
# Returns the euclidean distance calculated as the norm of the difference of their coordinates

    return np.linalg.norm(X1.getLoc()-X2.getLoc())

In [7]:
def usefulCenters(point, centers):
# Calculates the useful nearest centers of a given point
# Receives a point class variable and a list of center class variables
# Returns a list of center class variables representing the useful nearest centers
# NOTE: INEFFICIENT CODE --> REVISIT

    UNClist = []
    
    # Loop through centers and identify useful centers for given point
    for i in range(len(centers)):
        disPC = distance(point, centers[i])
        useless = False
        
        # Loop through other centers to determine if center i is useless
        # NOTE: INEFFICIENT CODE
        for j in range(len(centers)):
            if i != j:
                disPCx = distance(point, centers[j])
                disCCx = distance(centers[i], centers[j])
                if disPCx < disPC and disCCx < disPC:
                    useless = True
                    
        if useless == False:
            UNClist.append(centers[i])
    
    return UNClist

In [8]:
def UNCinit(points, maxCenters):
# Initialises the K-Means center with the Useful Nearest Center algorithm
# Receives a list of point class variables and a maxumum number of centers
# Returns the list of initial center class variables

    centerList = []

    # Loop through points and identify point with lowest first dimension value to be the first center
    minDimension = points[0].getLoc()[0]
    startPoint = points[0]
    startPointIndex = 0
    for i in range(points):
        firstDimension = points[i].getLoc()[0]
        if firstDimension < minDimension:
            minDimension = firstDimension
            startPoint = points[i]
            startPointIndex = i
            
    # Instantiate a center at the given location, and remove the point from the list
    centerCount = 1
    centerList.append(center(startPoint.getLoc(),centerCount))
    del points[startPointIndex]
    
    # Now need to loop until maximum number of centers is achieved
    while len(centerList) < maxCenters:
        
        # Calculate next optimal center
        maxFormula = 0
        maxPoint = 0
        maxPointIndex = 0
        for i in range(points):
            UNCi = usefulCenters(point[i], centerList)
            
            # Need to loop through useful nearest centers and calculate required distances to i
            maxDist = 0
            sumDist = 0
            sumLnDist = 0
            for j in UNCi:
                dist = distance(points[i],j)
                if dist > maxDist:
                    maxDist = dist
                sumDist += dist
                sumLnDist += np.log(dist)
            avgDist = sumDist/len(UNCi)
            
            # Calculate the formula of Ismkahn (2017) and compare to previous max
            formula = (avgDist/maxDist)*sumLnDist
            if formula > maxFormula:
                maxFormula = formula
                maxPoint = points[i]
                maxPointIndex = i

        # Instantiate a center at the given location, and remove the point from the list
        centerCount += 1
        centerList.append(center(maxPoint.getLoc(), centerCount))
        del points[maxPointIndex]
        
    return centerList

## SSEDM, Gain/Cost, & Adjacency functions

### SSEDM

In [10]:
def SSEDMi(center):
# Function to calculate the sum of squared euclidean distances from the mean (center) of the given cluster
# Function receives a center class object
# Function returns the calculated SSEDM for that center

    points = center.getPoints()
    SSEDMi = 0
    
    # Loop through the cluster's points to calculate their distances to the mean
    for i in points:
        SSEDMi += distance(point, center) ** 2
    
    return SSEDMi

In [11]:
def SSEDM(centerList):
# Function to calculate the sum of squared euclidean distances from the mean (center) of all clusters
# Function receives a list of center class objects
# Function returns the sum of calculated SSEDM for every cluster

    SSEDM = 0
    
    for i in centerList:
        SSEDM += SSEDMi(i)
    
    return SSEDM

### Cost calculation
the approximated cost of removing the cluster Sj is the increase in the total SSEDM over all clusters, SSEDM(S), after Sj is removed

In [13]:
def cost(removalCenter):
# Function to approximate the cost of removing the center from the overall solution
# Receives a center class object
# Returns the approximated cost of the removal

    sumSecondCenter = 0
    for i in removalCenter.getPoints():
        sumSecondCenter += i.disToSecond()**2
    
    return SSEDMi(removalCenter) - sumSecondCenter

### Gain calculation

In [14]:
def gain(splitCenter, alpha = 3/4):
# Function to approximate the gain of splitting one cluster into two clusters
# Receives a center class object, an alpha value which defaults to 3/4
# Returns the approximated gain for the split

    return alpha * SSEDMi(splitCenter)

### Adjacency

In [15]:
def adjacent(centerI, centerJ):
# Function to determine if two centers are adjacent according to Ismkahn's definition
# Receives two center class objects
# Returns True if centerJ is adjacent to centerI
# Returns False otherwise

    for i in centerI.getPoints():
        if i.getSecondCenter() == centerJ:
            return True
    
    return False

In [16]:
def strongAdjacent(centerI, centerJ):
# Function to determine if two centers are strongly adjacent according to Ismkahn's definition
# Receives two center class objects
# Returns True if centerJ is strongly adjacent to centerI
# Returns False otherwise

    if adjacent(centerI, centerJ) == True and adjacent(centerJ, centerI) == True:
        return True
    
    return False

In [17]:
def getAdjacentCenters(centerList, center):
# Generates a list of all centers in centerList which are adjacent to center
# Receives a list of center class objects and one center class object index to be checked
# Returns a list of centers adjacent to the center at centerIndex
    
    adj = []
    for i in range(len(centerList)):
        if centerList[i].getID() != center.getID():
            if adjacent(center, centerList[i]) == True:
                adj.append(centerList[i])
    
    return adj

### Affected points  
Definition: in solution S, point P is an affected point of center Ci iff the first or second nearest center of P is Ci

In [18]:
def affectedPointsInit(point, center):
# Determines if point is an affected point of center
# For the special T-K-means iteration 1 case, where we have not yet updated global point attributes but where we have moved the center
# Receives point and center class objects and a list of all centers
# Returns True if the point is affected
# Returns False otherwise
    
    second = point.disToSecond()
    new = distance(point, center)
    if new < second:
        return True
    
def affectedPoints(point, center):
# Determines if point is an affected point of center
# Receives point and center class objects and a list of all centers
# Returns True if the point is affected
# Returns False otherwise
    
    if point.getCenter() == center or point.getSecondCenter() == center:
        return True
    
    return False

## Topical K-means algorithm  
As applied to a set where the center Cj of cluster Sj is changed to be a random point in Si which has center Ci (i.e. Sj has been deleted and Si has been split)

In [ ]:
def TKmeans(centerList, deletedIndex, splitIndex):
    deletedCenter = centerList[deletedIndex]
    splitCenter = centerList[splitIndex]
    
# Phase 0: move the center
    # Define AC-Adjacent as the adjacent centers of Cj before removal
    ## QUESTION --> CAN Cj AND Ci BE ADJACENT???
    ACAdj = getAdjacentCenters(centerList, deletedCenter)
    
    # Define AP (affected points) as the affected points of Cj before removal
    AP = deletedCenter.getPoints()
    
    # Step 2, part 2 adds adjacent centers of the new Si and Sj in the first loop
    # This is equivalent to adding here adjacent centers of the old Si before splitting
    ACAdj.append(getAdjacentCenters(centerList, splitCenter))

    #### add center movement
    
    
# Phase 1: initialization  
    # Define AC (active centers) as a list of centers, holding only the new Ci and Cj
    AC = [deletedCenter, splitCenter]
    
    loop = 1
    
# Phase 2: identification
    # If AC is empty, go to end
    if len(AC) == 0:
        break
        
    # Add all adjacent centers of centers held in AC to AC-Adjacent
    if loop != 1:
        for i in AC:
            ACAdj.append(getAdjacentCenters(centerList, i))
    
    # Define Potential-AC as an empty list
    PAC = []
    
    # Add all affected points of centers in AC to AP
    if loop == 1:
        AP.append(affectedPointsInit())
    else:
        AP.append(affectedPoints())
        

# Phase 3: updating
    # For each point P in AP:
        # Update the first and second closest centers of P, considering as candidates centers held in AC and AC-Adjacent
        # If P’s closest center changes from Cx to Cy, add Cx and Cy to Potential-AC
    # Update all centers (re-calculate centroid of clusters)
    # Define AC as Potential-AC
    # If AC is not empty, return to Phase 2, else end
    loop += 1



## Iterative K-means -/+ algorithm

## Trials

In [ ]:
def plotkmeans(data,k,trials,iterations):
# Code from H440 by John Iacono
    
    km,pwkm=repeatkmeans(data,k,trials,iterations)
    nx=[]
    ny=[]
    sizes=[]
    cols=[]
    cm=matplotlib.cm.get_cmap('turbo')
    for m,P,c in zip(km,pwkm,[cm(i/len(km)) for i in range(len(km))]):
        nx.append(m[0])
        ny.append(m[1])
        cols.append(c)
        sizes.append(200)
        nx.extend(data[p][0] for p in P)
        ny.extend(data[p][1] for p in P)
        cols.extend([c]*len(P))
        sizes.extend([15]*len(P))

    fig, ax = plt.subplots(figsize=(20, 15))
    ax.scatter(nx, ny,sizes,cols) 

data=twodnormal( (8,4) ,1,20)+twodnormal( (1,1) ,2,60)+twodnormal( (3,10) ,1,30)+twodnormal( (9,0) ,0.2,20)
plotkmeans(data,4,100,20)

### Performance testing on generated data

### Performance testing on real-world data